In [1]:
import aocd
import math
import networkx as nx
import numpy as np
from enum import IntFlag

In [2]:
class Blizzard(IntFlag):
    UP = 1
    DOWN = 2
    LEFT = 4
    RIGHT = 8

def parse_blizzard(s):
    return {'^': Blizzard.UP, 'v': Blizzard.DOWN, '<': Blizzard.LEFT, '>': Blizzard.RIGHT, '.': Blizzard(0)}[s]

In [3]:
lines = aocd.get_data(day=24, year=2022).splitlines()
grid = np.array([[parse_blizzard(c) for c in line[1:-1]] for line in lines[1:-1]])

lx, ly = grid.shape
cycle_length = math.lcm(lx, ly)

start = -1, 0
goal = lx, ly-1

G = nx.grid_2d_graph(lx, ly)
G.add_edge(start, (0, 0))
G.add_edge(goal, (lx-1, ly-1))

In [4]:
blizzard_states = np.zeros((cycle_length, lx, ly), dtype=int)
blizzard_states[0] = grid

for step in range(1, cycle_length):
    for (x, y), cell in np.ndenumerate(blizzard_states[step-1]):
        if cell & Blizzard.UP:
            blizzard_states[step, (x-1) % lx, y] |= Blizzard.UP
        if cell & Blizzard.DOWN:
            blizzard_states[step, (x+1) % lx, y] |= Blizzard.DOWN
        if cell & Blizzard.LEFT:
            blizzard_states[step, x, (y-1) % ly] |= Blizzard.LEFT
        if cell & Blizzard.RIGHT:
            blizzard_states[step, x, (y+1) % ly] |= Blizzard.RIGHT

In [6]:
def step_bfs(start, goal, n=0, nodes=None):
    nodes = nodes if nodes else [start]
    next_nodes = set()

    for node in nodes:
        for neighbor in list(G[node]) + [node]:
            if neighbor == goal:
                return n+1
            if neighbor == start or blizzard_states[(n+1) % cycle_length, neighbor[0], neighbor[1]] == 0:
                next_nodes.add(neighbor)
    return step_bfs(start, goal, n+1, next_nodes)

there       = step_bfs(start, goal)
back_again  = step_bfs(goal, start, n=there)
there_again = step_bfs(start, goal, n=back_again)

print("Part 1:", there)
print("Part 2:", there_again)   

Part 1: 249
Part 2: 735
